### Day 7
#### part A

In [1]:
import re
import numpy as np

In [2]:
infile="07a_input.txt"

In [3]:
with open(infile) as f:
    lines=f.readlines()

In [4]:
lines=[line.replace("\n","") for line in lines]
lines[:5]

['drab tan bags contain 4 clear gold bags.',
 'vibrant lime bags contain 3 faded gold bags, 3 plaid aqua bags, 2 clear black bags.',
 'pale lime bags contain 1 dim salmon bag, 5 faded salmon bags, 1 dim turquoise bag.',
 'dull gray bags contain 1 striped gold bag, 1 vibrant yellow bag.',
 'light fuchsia bags contain 4 light lavender bags, 5 faded olive bags, 4 plaid cyan bags, 1 striped tomato bag.']

split each line into "outer color" bag, a list of allowed "inner color" bags and their "counts"

In [5]:
colors_outer=[]
colors_inner=[]
counts_inner=[]

for line in lines:
    outer, inner=line.split("contain")
    
    # outer
    curcol_outer, _ =outer.split(" bags",maxsplit=1)
    colors_outer.append(curcol_outer)
    
    # inner
    pattern=r"(?P<count>\d) (?P<color>\w+ \w+) bags?" # example hit: 1 shiny brown bag

    iterable=re.finditer(pattern, inner)
    curcol_inner=[]
    curcount_inner=[]
    for match in iterable:
        curcount_inner.append(int(match.group("count"))) # ex: 1
        curcol_inner.append(match.group("color")) # ex: shiny brown
    
    counts_inner.append(curcount_inner)
    colors_inner.append(curcol_inner)
    

In [6]:
# show how the data structure looks now
display(lines[:3])
print("outer:",colors_outer[:3])
print("inner count:",counts_inner[:3])
print("inner color:",colors_inner[:3])

['drab tan bags contain 4 clear gold bags.',
 'vibrant lime bags contain 3 faded gold bags, 3 plaid aqua bags, 2 clear black bags.',
 'pale lime bags contain 1 dim salmon bag, 5 faded salmon bags, 1 dim turquoise bag.']

outer: ['drab tan', 'vibrant lime', 'pale lime']
inner count: [[4], [3, 3, 2], [1, 5, 1]]
inner color: [['clear gold'], ['faded gold', 'plaid aqua', 'clear black'], ['dim salmon', 'faded salmon', 'dim turquoise']]


convert colors to numbers (indexing)

In [7]:
converter=dict(zip(colors_outer,range(len(colors_outer))))
converter

{'drab tan': 0,
 'vibrant lime': 1,
 'pale lime': 2,
 'dull gray': 3,
 'light fuchsia': 4,
 'drab gold': 5,
 'dim red': 6,
 'striped orange': 7,
 'shiny violet': 8,
 'faded olive': 9,
 'dotted lime': 10,
 'drab magenta': 11,
 'dull black': 12,
 'posh turquoise': 13,
 'muted purple': 14,
 'striped black': 15,
 'drab brown': 16,
 'pale purple': 17,
 'dark turquoise': 18,
 'clear lavender': 19,
 'bright coral': 20,
 'posh coral': 21,
 'drab lime': 22,
 'striped brown': 23,
 'wavy bronze': 24,
 'dim black': 25,
 'posh yellow': 26,
 'pale violet': 27,
 'mirrored bronze': 28,
 'vibrant silver': 29,
 'dark brown': 30,
 'wavy green': 31,
 'light beige': 32,
 'wavy crimson': 33,
 'clear tan': 34,
 'posh lime': 35,
 'pale blue': 36,
 'wavy silver': 37,
 'striped beige': 38,
 'dim gold': 39,
 'vibrant teal': 40,
 'light tan': 41,
 'dark red': 42,
 'posh salmon': 43,
 'mirrored yellow': 44,
 'dark fuchsia': 45,
 'bright bronze': 46,
 'posh silver': 47,
 'dotted salmon': 48,
 'dark orange': 49,
 'b

In [8]:
# inner bag colors as indices
idxs_cols_inner=[]
for curcols in colors_inner:
    curidxs=[converter[c] for c in curcols]
    idxs_cols_inner.append(curidxs)
idxs_cols_inner[:3]

[[575], [453, 367, 387], [198, 120, 160]]

create color transition matrix: outer -> inner
* Rows: outer color, Columns: inner color
* one row stands for: all the possible inner colors that can be contained in that specific outer color
* To check whether a specific inner color can be contained in an outer bag: check the corresponding column if it contains a nonzero element


In [9]:
numcolors=len(colors_outer)
numcolors

594

In [10]:
Tmat=np.zeros((numcolors,numcolors)) # transition matrix

In [11]:
for idx in range(len(colors_inner)):
    Tmat[idx,idxs_cols_inner[idx]]=counts_inner[idx]

shiny gold bag
* sum up in which bags it can be contained after 1,2,3,... iterations (iteration here means: 1,2,3 bags mamoushka style packed into each other)

In [12]:
goldidx=converter["shiny gold"]
goldidx

327

In [13]:
# number of bags that contain directly a shiny gold bag: (for exploration)
len(np.nonzero(Tmat[:,goldidx])[0])

9

In [14]:
# number of bags that contain a shiny gold bag after 0,1,2,3,... iterations
Tn=np.identity(numcolors)
idxs_outer=np.array([])
for idx in range(numcolors):
    Tn=Tmat.dot(Tn)
    idxs_outer=np.concatenate((idxs_outer,(np.nonzero(Tn[:,goldidx])[0])))
    print(idx,",  ",np.sum(Tn[:,goldidx]))
    if np.sum(Tn[:,goldidx])==0:
        break # we'll never get a shiny gold bag further inside

0 ,   29.0
1 ,   363.0
2 ,   2607.0
3 ,   15247.0
4 ,   69932.0
5 ,   303486.0
6 ,   858136.0
7 ,   1729542.0
8 ,   2492780.0
9 ,   5314344.0
10 ,   4776696.0
11 ,   5867472.0
12 ,   12665088.0
13 ,   14307840.0
14 ,   33868800.0
15 ,   19353600.0
16 ,   0.0


In [15]:
# total number of different outer bags
len(np.unique(idxs_outer))

370

#### exploration: time profiling

In [16]:
%load_ext line_profiler

In [17]:
def test_fct():
    # the idx finding function copied. uses global vars
    Tn=np.identity(numcolors)
    idxs_outer=np.array([])

    for idx in range(numcolors):
        Tn=Tmat.dot(Tn)
        idxs_outer=np.concatenate((idxs_outer,(np.nonzero(Tn[:,goldidx])[0])))
        print(idx,",  ",np.sum(Tn[:,goldidx]))
        if np.sum(Tn[:,goldidx])==0:
            break # we'll never get a shiny gold bag further inside

In [18]:
%lprun -f test_fct test_fct()

0 ,   29.0
1 ,   363.0
2 ,   2607.0
3 ,   15247.0
4 ,   69932.0
5 ,   303486.0
6 ,   858136.0
7 ,   1729542.0
8 ,   2492780.0
9 ,   5314344.0
10 ,   4776696.0
11 ,   5867472.0
12 ,   12665088.0
13 ,   14307840.0
14 ,   33868800.0
15 ,   19353600.0
16 ,   0.0


#### exploration: regex with groups

In [19]:
line=lines[1]
outer, inner=line.split("contain")
print(inner,"\n")

pattern=r"(?P<count>\d) (?P<color>\w+ \w+) bags?"

# return all detected groups as list of tuples
x=re.findall(pattern,inner)
print(x,"\n")

# return all detected match objects and access the strings in a loop
iterable=re.finditer(pattern, inner)
for m in iterable:
    print(m.group(0)) # full match
    print(m.group("color"), m.group("count"))
    print(m.groupdict(),"\n") # alternative

 3 faded gold bags, 3 plaid aqua bags, 2 clear black bags. 

[('3', 'faded gold'), ('3', 'plaid aqua'), ('2', 'clear black')] 

3 faded gold bags
faded gold 3
{'count': '3', 'color': 'faded gold'} 

3 plaid aqua bags
plaid aqua 3
{'count': '3', 'color': 'plaid aqua'} 

2 clear black bags
clear black 2
{'count': '2', 'color': 'clear black'} 



#### Part B
now the shiny gold bag is outside

In [20]:
# all bags that are contained in a shiny gold bag
Tn=np.identity(numcolors)

counter=0 # count number of bags inside the golden one

for idx in range(numcolors): # after 0,1,2,... packing iterations
    Tn=Tmat.dot(Tn)
    curcount=np.sum(Tn[goldidx,:])
    print(idx,",  ",curcount)
    counter+=curcount
    if np.max(Tn)==0:
        break # we'll never get a shiny gold bag further inside since whole matrix is 0

0 ,   7.0
1 ,   70.0
2 ,   701.0
3 ,   2245.0
4 ,   1574.0
5 ,   7850.0
6 ,   17100.0
7 ,   0.0
8 ,   0.0
9 ,   0.0
10 ,   0.0
11 ,   0.0
12 ,   0.0
13 ,   0.0
14 ,   0.0
15 ,   0.0
16 ,   0.0
17 ,   0.0
18 ,   0.0
19 ,   0.0
20 ,   0.0
21 ,   0.0
22 ,   0.0
23 ,   0.0
24 ,   0.0


In [21]:
# total nr of bags in side the shiny golden one
counter

29547.0